<a href="https://colab.research.google.com/github/astriwlndr/Tubes_SistemCerdas/blob/main/Main_ANN_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("drive/My Drive/SISTEM CERDAS/TUBES")

In [ ]:
# Memanggil Dependencies
import pandas as pd 
import numpy as np 
import random 
import matplotlib.pyplot as plt 
import sklearn
import keras
import tensorflow as tf      

from keras.models import Sequential 
from keras.callbacks import EarlyStopping 
from keras.callbacks import ModelCheckpoint 
from keras.models import load_model

from numpy import ndarray 
from sklearn.metrics import confusion_matrix, roc_curve, auc, roc_auc_score, accuracy_score, classification_report

RANDOM_SEED = 27

In [ ]:
# Load dataset
df = pd.read_csv('input/diabetesPreProcessed.csv')

# Mengambil data pada kolom Outcome ke variabel target sebagai numpy Array
dftarget = df['Outcome']
target = dftarget.to_numpy()

# Menghapus kolom Outcome untuk mengambil data seluruh features
df = df.drop(columns=['Outcome'])

# Seluruh data features disimpan ke dalam variabel data sebagai numpy Array
data = df.to_numpy()

In [ ]:
# Standarisasi Data
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

In [ ]:
# Split Data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=RANDOM_SEED)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(604, 8) (152, 8) (604,) (152,)


In [ ]:
# Training Neural Network
import time
start = time.perf_counter()
np.random.seed(RANDOM_SEED)

# define model
model = Sequential()
model.add(Dense(16, input_dim=8, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))
tf.keras.optimizers.SGD(lr=0.02)
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

# simple early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

# fit model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2000, verbose=0, callbacks=[es, mc])

# load the saved model
saved_model = load_model('best_model.h5')

# evaluate the model
_, train_acc = saved_model.evaluate(X_train, y_train, verbose=0)
_, test_acc = saved_model.evaluate(X_test, y_test, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))

elapsed = time.perf_counter() - start
print('Elapsed %.3f seconds.' % elapsed)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)



Epoch 00001: val_accuracy improved from -inf to 0.53947, saving model to best_model.h5

Epoch 00002: val_accuracy improved from 0.53947 to 0.61184, saving model to best_model.h5
Epoch 00002: early stopping
Train: 0.621, Test: 0.612
Elapsed 1.857 seconds.


In [ ]:
# Testing Neural Network
pred = saved_model.predict(X_test)
classsificationPreds = np.array([])
for i in pred:
    if i < 0.5:
        classsificationPreds = np.append(classsificationPreds, [0])
    else:
        classsificationPreds = np.append(classsificationPreds, [1])

In [ ]:
# Confusion Matrix untuk set testing
print(confusion_matrix(y_test, classsificationPreds))
print(classification_report(y_test, classsificationPreds))

[[73 32]
 [27 20]]
              precision    recall  f1-score   support

           0       0.73      0.70      0.71       105
           1       0.38      0.43      0.40        47

    accuracy                           0.61       152
   macro avg       0.56      0.56      0.56       152
weighted avg       0.62      0.61      0.62       152

